In [1]:
%pip install pypdf
%pip install -q transformers einops accelerate langchain bitsandbytes
%pip install -U sentence-transformers
%pip install llama-index
%pip install llama-index-llms-huggingface
%pip install llama-index-llms-huggingface-api
%pip install langchain-huggingface
%pip install llama-index-embeddings-langchain

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [1]:
from llama_index.core import VectorStoreIndex, SimpleDirectoryReader,ServiceContext
from llama_index.llms.huggingface import HuggingFaceLLM
from llama_index.core.prompts.prompts import SimpleInputPrompt

/home/kartik/miniconda3/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
documents = SimpleDirectoryReader('/home/kartik/books').load_data()
documents

[Document(id_='40499e07-7e29-43d0-8124-585c671c846d', embedding=None, metadata={'page_label': '1', 'file_name': 'Personal Fitness and Wellness - fitness-handbook.pdf', 'file_path': '/home/kartik/books/Personal Fitness and Wellness - fitness-handbook.pdf', 'file_type': 'application/pdf', 'file_size': 148588, 'creation_date': '2025-04-04', 'last_modified_date': '2025-04-04'}, excluded_embed_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], excluded_llm_metadata_keys=['file_name', 'file_type', 'file_size', 'creation_date', 'last_modified_date', 'last_accessed_date'], relationships={}, metadata_template='{key}: {value}', metadata_separator='\n', text_resource=MediaResource(embeddings=None, data=None, text=' 1 \nTHE COMPLETE \nFITNESS \n \n \n \n \n \n \n \n \nHANDBOOK \n \n \n \n \n \n \n \n \n \n \nTable of Contents \n \nIntroduction……………………………………………………………………… 3 \n \n', path=None, url=None, mimetype=None), image_resource=No

In [3]:
system_prompt = """You are an assistant designed to help with question-answering tasks. Use the following pieces of retrieved context to provide a concise and accurate answer to the question. Keep your response to a maximum of ten sentences.
                    If you don't know the answer, just say "I do not know." Don't make up an answer. Answer without repeating the labels or unnecessary text.."""
# Default format
query_wrapper_prompt = SimpleInputPrompt("<|user|>{query_str}<|Assistent|")

In [13]:
import torch

llm = HuggingFaceLLM(
    context_window=4096,
    max_new_tokens=256,
    generate_kwargs={"temperature": 0.5, "do_sample": True},
    system_prompt = system_prompt,
    query_wrapper_prompt = query_wrapper_prompt,
    tokenizer_name="meta-llama/Llama-2-7b-chat-hf",
    model_name="meta-llama/Llama-2-7b-chat-hf",
    device_map="auto",
    tokenizer_kwargs={"trust_remote_code": True},
    model_kwargs={"torch_dtype": torch.float16, "trust_remote_code": True}
)

Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00, 34.09it/s]
Some parameters are on the meta device because they were offloaded to the disk and cpu.


In [14]:
from langchain_huggingface import HuggingFaceEmbeddings
from llama_index.core import ServiceContext
from llama_index.embeddings.langchain import LangchainEmbedding

embd_model = LangchainEmbedding(HuggingFaceEmbeddings(model_name = "sentence-transformers/all-mpnet-base-v2"))

In [16]:
from llama_index.core import Settings

# Configure settings with LLM and embedding model
Settings.llm = llm  # Your LLM instance (e.g., OpenAI, HuggingFace)
Settings.embed_model = embd_model  # Your embedding model
Settings.chunk_size = 1024  # Chunk size setting

In [17]:
valid_documents = []
for doc in documents:
    if hasattr(doc, "text") and isinstance(doc.text, str) and doc.text.strip():
        valid_documents.append(doc)

# Now use valid_documents to create your index
index = VectorStoreIndex.from_documents(valid_documents)

In [18]:
index

In [19]:
query_engine =  index.as_query_engine()

In [20]:
response = query_engine.query("tell me some exersises for shoulders")

KeyboardInterrupt: 